# Como construir um sistema de recomendação para dados de compra (passo a passo)
* Descrição: uma documentação sobre a construção de modelos de filtragem colaborativa para recomendar produtos aos clientes
* Link: https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
* Autor: Moorissa Tjokro

## Declaração do problema
Neste desafio de dados, estamos construindo modelos de filtragem colaborativa para recomendar itens de produtos. As etapas a seguir têm como objetivo recomendar aos usuários seus 10 principais itens para colocar na cesta. A saída final será um arquivo csv na pasta `output` e uma função que procura uma lista de recomendações com base em um usuário especificado:
* Entrada: usuário - ID do cliente
* Retorna: lista classificada de itens (IDs de produto), que o usuário mais provavelmente deseja colocar em sua "cesta" (vazia)

## 1. Módulos de importação
* `pandas` e` numpy` para manipulação de dados
* `turicreate` para realizar a seleção e avaliação do modelo
* `sklearn` para dividir os dados no conjunto de treino e teste

In [ ]:
%load_ext autoreload
%autoreload 2
!pip install turicreate

import pandas as pd
import numpy as np
import time
import turicreate as tc
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

#import sys
#sys.path.append("..")
#!pip install scripts
#import scripts.data_layer as data_layer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Carregar dados
Dois conjuntos de dados são usados ​​neste exercício, que podem ser encontrados na pasta `data`:
* `recommend_1.csv` consistindo em uma lista de 1000 IDs de cliente para recomendar como saída
* `trx_data.csv` consistindo em transações do usuário

O formato é o seguinte.

## 3. Preparação de dados
* Nosso objetivo aqui é dividir cada lista de itens na coluna `produtos` em linhas e contar o número de produtos comprados por um usuário

In [ ]:
data=pd.read_csv('/content/df_grouped.csv',sep=';')
data=data.rename(columns={'userId':'customerId','counts':'purchase_count'})
data.drop(['Unnamed: 0'],axis=1,inplace=True) 
data['productId'] = data['productId'].apply(np.int64)
data=data.loc[:20000,:]
data_original=data.copy()

### 3.1. Crie dados com usuário, item e campo de destino
* Esta tabela será uma entrada para nossa modelagem posterior
    * Nesse caso, nosso usuário é `customerId`,` productId` e `purchase_count`

In [ ]:
print(data.shape)
data.head()

(20001, 3)


,customerId,productId,purchase_count
0,1,81572,1
1,2,81572,2
2,29,81065,1
3,29,81730,1
4,29,83484,1


### 3.2. Criar dummy
* Dummy para marcar se um cliente comprou aquele item ou não.
* Se alguém compra um item, `purchase_dummy` é marcado como 1
* Por que criar um dummy em vez de normalizá-lo?
    * Normalizar a contagem de compras, digamos por cada usuário, não funcionaria porque os clientes podem ter frequências de compra diferentes e não têm o mesmo gosto
    * No entanto, podemos normalizar os itens por frequência de compra para todos os usuários, o que é feito na seção 3.3. abaixo de.

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [ ]:
data_dummy = create_data_dummy(data)
data_dummy

,customerId,productId,purchase_count,purchase_dummy
0,1,81572,1,1
1,2,81572,2,1
2,29,81065,1,1
3,29,81730,1,1
4,29,83484,1,1
...,...,...,...,...
19996,127238,10437,1,1
19997,127238,68857,1,1
19998,127238,71346,1,1
19999,127238,81572,1,1


### 3.3. Normalizar os valores dos itens entre os usuários
* Para fazer isso, normalizamos a frequência de compra de cada item entre os usuários, primeiro criando uma matriz de item de usuário da seguinte maneira

In [ ]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix

productId,1,2,10056,10057,10072,10083,10084,10098,10104,10128,10129,10171,10175,10195,10200,10202,10208,10225,10240,10241,10262,10263,10276,10305,10314,10315,10327,10360,10382,10432,10437,10439,10440,10447,10448,10451,10643,10649,10650,10654,...,104210,104213,104216,104217,104218,104220,104221,104222,104223,104229,104311,104312,104373,104383,104391,104471,104506,104511,104541,104696,104766,104838,104871,104872,104922,104936,105053,105061,105080,105092,105111,105116,105222,105516,105566,105605,968503,968504,1067221,1067222
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
(df_matrix.shape)

(8982, 6910)

In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm

(8982, 6910)


productId,1,2,10056,10057,10072,10083,10084,10098,10104,10128,10129,10171,10175,10195,10200,10202,10208,10225,10240,10241,10262,10263,10276,10305,10314,10315,10327,10360,10382,10432,10437,10439,10440,10447,10448,10451,10643,10649,10650,10654,...,104210,104213,104216,104217,104218,104220,104221,104222,104223,104229,104311,104312,104373,104383,104391,104471,104506,104511,104541,104696,104766,104838,104871,104872,104922,104936,105053,105061,105080,105092,105111,105116,105222,105516,105566,105605,968503,968504,1067221,1067222
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm

(2332, 3)


,customerId,productId,scaled_purchase_freq
80842,247,10128,1.0
81216,785,10128,0.0
81629,1224,10128,0.0
83805,3501,10128,0.0
84051,3752,10128,0.0
...,...,...,...
52741864,110656,96161,0.0
52742110,119652,96161,0.0
52742120,119883,96161,0.0
52742269,126333,96161,0.0


#### Defina uma função para normalizar dados




In [ ]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* Podemos normalizar o histórico de compras, de 0 a 1 (com 1 sendo o maior número de compras para um item e 0 sendo 0 contagem de compras para aquele item).

## 4. Divida o conjunto de dados em treino e teste
* Dividir os dados em conjuntos de treinamento e teste é uma parte importante da avaliação da modelagem preditiva, neste caso, um modelo de filtragem colaborativa. Normalmente, usamos uma parte maior dos dados para treinamento e uma parte menor para teste.
* Usamos a proporção 80:20 para o tamanho do nosso conjunto de treino e teste.
* Nossa parte de treinamento será usada para desenvolver um modelo preditivo, enquanto a outra para avaliar o desempenho do modelo.
* Agora que temos três conjuntos de dados com contagens de compra, compras dummy e contagens de compra normalizadas, gostaríamos de dividir cada um.

In [ ]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(16000, 3) (4001, 3)


In [ ]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [ ]:
train_data

customerId,productId,purchase_count
114482,10057,1
4186,91585,1
15770,81572,1
60277,84086,1
31807,90177,1
783,99189,1
821,78918,1
941,78100,1
2544,75694,1
3301,92773,1


In [ ]:
test_data

customerId,productId,purchase_count
3930,74349,1
74360,94142,1
2793,82148,1
2560,57845,1
808,83182,1
73990,68919,1
6480,69522,1
1991,82958,1
125243,68744,1
5040,65537,1


#### Define a função `split_data` para dividir os dados de treino e teste

In [ ]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Modelo básico
Antes de executar uma abordagem mais complicada, como filtragem colaborativa, gostaríamos de usar um modelo básico para comparar e avaliar modelos. Esse modelo normalmente usa uma abordagem muito simples, as técnicas usadas além dessa abordagem devem ser escolhidas se mostrarem precisão e complexidade relativamente melhores.

### 5.1. Usando um modelo de popularidade como modelo básico
* O modelo de popularidade usa os itens mais populares para recomendação. Esses itens são produtos com o maior número de vendas entre os clientes.
* Usamos a biblioteca `turicreate` para executar e avaliar os modelos de filtragem básico e colaborativo abaixo
* Os dados de treinamento são usados ​​para a seleção do modelo

#### Usando contagens de compra

In [ ]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(data_original[user_id])
#users_to_recommend = list(userId)
n_rec = 10 # number of items to recommend
n_display = 30

In [ ]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 16000 observations with 8103 users and 6107 items.

Data prepared in: 0.030537s

16000 observations to process; with 6107 unique items.

In [ ]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/20001 queries. users per second: 20756.8

recommendations finished on 2000/20001 queries. users per second: 20259.9

recommendations finished on 3000/20001 queries. users per second: 20164.3

recommendations finished on 4000/20001 queries. users per second: 20043.4

recommendations finished on 5000/20001 queries. users per second: 19803.4

...

recommendations finished on 17000/20001 queries. users per second: 19123.1

recommendations finished on 18000/20001 queries. users per second: 19155.7

recommendations finished on 19000/20001 queries. users per second: 19054.7

recommendations finished on 20000/20001 queries. users per second: 19053.6

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     1      |   49653   |        2.0         |  1   |
|     1      |   49666   |        2.0         |  2   |
|     1      |   85255   |        2.0         |  3   |
|     1      |     2     |        2.0         |  4   |
|     1      |   48887   | 1.3333333333333333 |  5   |
|     1      |   36365   |        1.2         |  6   |
|     1      |   54833   |        1.2         |  7   |
|     1      |   80811   |       1.125        |  8   |
|     1      |   74095   | 1.0909090909090908 |  9   |
|     1      |   66005   | 1.0714285714285714 |  10  |
|     2      |   49653   |        2.0         |  1   |
|     2      |   49666   |        2.0         |  2   |
|     2      |   85255   |        2.0         |  3   |
|     2      |     2     |        2.0         |  4   |
|     2      |   48887   | 1.3333333333333333 |  5   |
|     2   

#### Definir uma função `model` para a seleção do modelo

In [ ]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(userId)
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Usando dummy

In [ ]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 16000 observations with 8145 users and 6055 items.

Data prepared in: 0.035798s

...

recommendations finished on 17000/20001 queries. users per second: 19319.3

recommendations finished on 18000/20001 queries. users per second: 19344.7

recommendations finished on 19000/20001 queries. users per second: 19378.3

recommendations finished on 20000/20001 queries. users per second: 19374.7

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     1      |   91353   |  1.0  |  1   |
|     1      |   94331   |  1.0  |  2   |
|     1      |   94145   |  1.0  |  3   |
|     1      |   75694   |  1.0  |  4   |
|     1      |   98439   |  1.0  |  5   |
|     1      |   87223   |  1.0  |  6   |
|     1      |   71532   |  1.0  |  7   |
|     1      |   70901   |  1.0  |  8   |
|     1      |   87170   |  1.0  |  9   |
|     1      |   87962   |  1.0  |  10  |
|     2      |   91353   |  1.0  |  1   |
|     2      |   94331   |  1.0  |  2   |
|     2      |   94145   |  1.0  |  3   |
|     2      |   75694   |  1.0  |  4   |
|     2      |   98439   |  1.0  |  5   |
|     2      |   87223   |  1.0  |  6   |
|     2      |   71532   |  1.0  |  7   |
|     2      |   70901   |  1.0  |  8   |
|     2      |   87170   |  1.0  |  9   |
|     2      |   87962   |  1.0  |  10  |
|     29     |   91353   |  1.0  |

#### Usando contagem normalizada

In [ ]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 1865 observations with 1718 users and 20 items.

Data prepared in: 0.008822s

1865 observations to process; with 20 unique items.

recommendations finished on 1000/20001 queries. users per second: 419639

...

recommendations finished on 17000/20001 queries. users per second: 231668

recommendations finished on 18000/20001 queries. users per second: 224218

recommendations finished on 19000/20001 queries. users per second: 218748

recommendations finished on 20000/20001 queries. users per second: 199191

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     1      |   79595   |         0.5          |  1   |
|     1      |   49644   |         0.5          |  2   |
|     1      |   49653   |         0.5          |  3   |
|     1      |   48887   |  0.3333333333333333  |  4   |
|     1      |   54833   | 0.16666666666666666  |  5   |
|     1      |   80811   |        0.125         |  6   |
|     1      |   74095   | 0.07142857142857142  |  7   |
|     1      |   80711   | 0.043478260869565216 |  8   |
|     1      |   81225   | 0.041666666666666664 |  9   |
|     1      |   87962   | 0.022727272727272728 |  10  |
|     2      |   79595   |         0.5          |  1   |
|     2      |   49644   |         0.5          |  2   |
|     2      |   49653   |         0.5          |  3   |
|     2      |   48887   |  0.3333333333333333  |  4   |
|     2      |   54833   | 0.16

#### Notas
* Depois de criar o modelo, previmos os itens de recomendação usando pontuações por popularidade. Como você pode ver para cada resultado do modelo acima, as linhas mostram os primeiros 30 registros de 1000 usuários com 10 recomendações. Esses 30 registros incluem 3 usuários e seus itens recomendados, junto com a pontuação e as classificações decrescentes.
* No resultado, embora diferentes modelos tenham diferentes listas de recomendações, a cada usuário é recomendada a mesma lista de 10 itens. Isso ocorre porque a popularidade é calculada considerando os itens mais populares de todos os usuários.
* Se for um exemplo de agrupamento abaixo, os produtos 132, 248, 37 e 34 são os mais populares (mais vendidos) entre os clientes. Usando suas contagens de compras divididas pelo número de clientes, vemos que esses produtos são comprados pelo menos 3 vezes em média no conjunto de transações de treinamento (o mesmo que a primeira medida de popularidade na variável `compra_conta`)

In [ ]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
49653    2.000000
2        2.000000
49666    2.000000
85255    2.000000
48887    1.333333
54833    1.200000
36365    1.200000
80811    1.125000
74095    1.090909
66005    1.071429
81225    1.047619
81572    1.045346
80711    1.043478
81774    1.022727
87962    1.022727
10128    1.019608
96161    1.010989
70781    1.009174
75694    1.003534
81773    1.003077
Name: purchase_count, dtype: float64

## 6. Modelo de Filtragem Colaborativa

* Na filtragem colaborativa, recomendamos itens com base em como usuários semelhantes compram itens. Por exemplo, se o cliente 1 e o cliente 2 compraram itens semelhantes, por exemplo, 1 comprei X, Y, Z e 2 comprei X, Y, recomendaríamos um item Z ao cliente 2.

* Para definir a similaridade entre os usuários, usamos as seguintes etapas:
    1. Crie uma matriz de item de usuário, onde os valores de índice representam IDs de clientes exclusivos e os valores de coluna representam IDs de produtos exclusivos
    
    2. Crie uma matriz de similaridade item a item. A ideia é calcular a semelhança de um produto com outro produto. Existem várias maneiras de calcular isso. Nas etapas 6.1 e 6.2, usamos co-seno e medida de similaridade de Pearson, respectivamente.
    
        * Para calcular a similaridade entre os produtos X e Y, observe todos os clientes que avaliaram esses dois itens. Por exemplo, tanto X quanto Y foram avaliados pelos clientes 1 e 2.
        * Em seguida, criamos dois vetores de itens, v1 para o item X e v2 para o item Y, no espaço do usuário de (1, 2) e, a seguir, encontramos o ângulo / distância `cosseno` ou` pearson` entre esses vetores. Um ângulo zero ou vetores sobrepostos com valor de cosseno de 1 significam similaridade total (ou por usuário, em todos os itens, há a mesma classificação) e um ângulo de 90 graus significaria cosseno de 0 ou nenhuma similaridade.
        
    3. Para cada cliente, prevemos sua probabilidade de comprar um produto (ou suas contagens de compra) para produtos que ele não comprou.
    
        * Para o nosso exemplo, calcularemos a classificação para o usuário 2 no caso do item Z (item alvo). Para calcular isso, pesamos a medida de similaridade calculada entre o item de destino e outros itens que o cliente já comprou. O fator de pesagem são as contagens de compras fornecidas pelo usuário aos itens já adquiridos por ele.
        * Em seguida, normalizamos essa soma ponderada com a soma das medidas de similaridade para que a classificação calculada permaneça dentro dos limites predefinidos. Assim, a classificação prevista para o item Z para o usuário 2 seria calculada usando medidas de similaridade.

* Embora eu tenha escrito scripts Python para todo o processo, incluindo encontrar semelhanças usando scripts Python (que podem ser encontrados na pasta `scripts`, podemos usar a biblioteca` turicreate` por enquanto para capturar diferentes medidas, como usar distância de `cosine` e` pearson` e avalie o melhor modelo.

### 6.1. Similaridade `Cosine`
* Similaridade é o cosseno do ângulo entre os 2 vetores dos vetores dos itens de A e B
* É definido pela seguinte fórmula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Quanto mais próximos os vetores, menor será o ângulo e maior será o cosseno


#### Usando contagem de compra

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 16000 observations with 8103 users and 6107 items.

Data prepared in: 0.030746s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 17000/20001 queries. users per second: 14354.4

recommendations finished on 18000/20001 queries. users per second: 14383.4

recommendations finished on 19000/20001 queries. users per second: 14358.8

recommendations finished on 20000/20001 queries. users per second: 14331.1

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     1      |   100951  | 0.10726732015609741  |  1   |
|     1      |   96161   | 0.10378742218017578  |  2   |
|     1      |   86634   | 0.09147787094116211  |  3   |
|     1      |   74612   | 0.09147787094116211  |  4   |
|     1      |   103600  | 0.09147787094116211  |  5   |
|     1      |   99857   | 0.09147787094116211  |  6   |
|     1      |   61357   | 0.09147787094116211  |  7   |
|     1      |   86010   | 0.09147787094116211  |  8   |
|     1      |   102129  | 0.09147787094116211  |  9   |
|     1      |   87133   | 0.07922214269638062  |  10  |
|     2      |   100951  | 0.21453464031219482  |  1   |
|     2      |   96161   | 0.20757484436035156  |  2   |
|     2      |   86634   | 0.18295574188232422  |  3   |
|     2      |   74612   | 0.18295574188232422  |  4   |
|     2      |   103600  | 0.18

#### Usando dummy

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 16000 observations with 8145 users and 6055 items.

Data prepared in: 0.035015s

...

recommendations finished on 17000/20001 queries. users per second: 14353.8

recommendations finished on 18000/20001 queries. users per second: 14363.7

recommendations finished on 19000/20001 queries. users per second: 14214.3

recommendations finished on 20000/20001 queries. users per second: 14157.6

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     1      |   96161   | 0.10425013303756714  |  1   |
|     1      |   100951  |  0.0752210021018982  |  2   |
|     1      |   65540   | 0.07244855165481567  |  3   |
|     1      |   96481   | 0.06796276569366455  |  4   |
|     1      |     2     | 0.06796276569366455  |  5   |
|     1      |   88203   | 0.06796276569366455  |  6   |
|     1      |   63632   | 0.05549138784408569  |  7   |
|     1      |   96558   | 0.05549138784408569  |  8   |
|     1      |   92618   | 0.05549138784408569  |  9   |
|     1      |   68011   | 0.05549138784408569  |  10  |
|     2      |   91353   |         0.0          |  1   |
|     2      |   94331   |         0.0          |  2   |
|     2      |   94145   |         0.0          |  3   |
|     2      |   75694   |         0.0          |  4   |
|     2      |   98439   |     

#### Usando contagem de compra normalizada

In [ ]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 1865 observations with 1718 users and 20 items.

Data prepared in: 0.006037s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 17000/20001 queries. users per second: 184622

recommendations finished on 18000/20001 queries. users per second: 188656

recommendations finished on 19000/20001 queries. users per second: 190583

recommendations finished on 20000/20001 queries. users per second: 192088

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     1      |   49644   | 0.025 |  1   |
|     1      |   49653   | 0.025 |  2   |
|     1      |   96161   |  0.0  |  3   |
|     1      |   74095   |  0.0  |  4   |
|     1      |   80711   |  0.0  |  5   |
|     1      |   10128   |  0.0  |  6   |
|     1      |   87962   |  0.0  |  7   |
|     1      |   81773   |  0.0  |  8   |
|     1      |   10437   |  0.0  |  9   |
|     1      |   70781   |  0.0  |  10  |
|     2      |   49644   | 0.025 |  1   |
|     2      |   49653   | 0.025 |  2   |
|     2      |   74095   |  0.0  |  3   |
|     2      |   80711   |  0.0  |  4   |
|     2      |   10128   |  0.0  |  5   |
|     2      |   87962   |  0.0  |  6   |
|     2      |   81773   |  0.0  |  7   |
|     2      |   10437   |  0.0  |  8   |
|     2      |   81572   |  0.0  |  9   |
|     2      |   70781   |  0.0  |  10  |
|     29     |   49644   | 0.025 |

### 6.2. Similaridade de `Pearson`
* Similaridade é o coeficiente de Pearson entre os dois vetores.
* É definido pela seguinte fórmula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Usando contagem de compra

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 16000 observations with 8103 users and 6107 items.

Data prepared in: 0.027411s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 17000/20001 queries. users per second: 11704.7

recommendations finished on 18000/20001 queries. users per second: 11750.3

recommendations finished on 19000/20001 queries. users per second: 11778.8

recommendations finished on 20000/20001 queries. users per second: 11768.5

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     1      |   49653   |        2.0         |  1   |
|     1      |   49666   |        2.0         |  2   |
|     1      |   85255   |        2.0         |  3   |
|     1      |     2     |        2.0         |  4   |
|     1      |   48887   | 1.3333333333333333 |  5   |
|     1      |   36365   |        1.2         |  6   |
|     1      |   54833   |        1.2         |  7   |
|     1      |   80811   | 1.1250000000000002 |  8   |
|     1      |   74095   | 1.0908674868670378 |  9   |
|     1      |   66005   | 1.0713959080832347 |  10  |
|     2      |   49653   |        2.0         |  1   |
|     2      |   49666   |        2.0         |  2   |
|     2      |   85255   |        2.0         |  3   |
|     2      |     2     |        2.0         |  4   |
|     2      |   48887   | 1.3333333333333333 |  5   |
|     2   

#### Usando compra dummy

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 16000 observations with 8145 users and 6055 items.

Data prepared in: 0.029416s

...

recommendations finished on 17000/20001 queries. users per second: 10517.5

recommendations finished on 18000/20001 queries. users per second: 10550.3

recommendations finished on 19000/20001 queries. users per second: 10570.4

recommendations finished on 20000/20001 queries. users per second: 10572.6

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     1      |   91353   |  0.0  |  1   |
|     1      |   94331   |  0.0  |  2   |
|     1      |   94145   |  0.0  |  3   |
|     1      |   75694   |  0.0  |  4   |
|     1      |   98439   |  0.0  |  5   |
|     1      |   87223   |  0.0  |  6   |
|     1      |   71532   |  0.0  |  7   |
|     1      |   70901   |  0.0  |  8   |
|     1      |   87170   |  0.0  |  9   |
|     1      |   87962   |  0.0  |  10  |
|     2      |   91353   |  0.0  |  1   |
|     2      |   94331   |  0.0  |  2   |
|     2      |   94145   |  0.0  |  3   |
|     2      |   75694   |  0.0  |  4   |
|     2      |   98439   |  0.0  |  5   |
|     2      |   87223   |  0.0  |  6   |
|     2      |   71532   |  0.0  |  7   |
|     2      |   70901   |  0.0  |  8   |
|     2      |   87170   |  0.0  |  9   |
|     2      |   87962   |  0.0  |  10  |
|     29     |   91353   |  0.0  |

#### Usando contagem normalizada de compra

In [ ]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 1865 observations with 1718 users and 20 items.

Data prepared in: 0.007396s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 17000/20001 queries. users per second: 144993

recommendations finished on 18000/20001 queries. users per second: 148339

recommendations finished on 19000/20001 queries. users per second: 146184

recommendations finished on 20000/20001 queries. users per second: 144660

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     1      |   79595   |         0.5          |  1   |
|     1      |   49644   |         0.5          |  2   |
|     1      |   49653   |         0.5          |  3   |
|     1      |   48887   | 0.33333333333333337  |  4   |
|     1      |   54833   | 0.16666666666666666  |  5   |
|     1      |   80811   | 0.12500000000000003  |  6   |
|     1      |   74095   | 0.07142857142857142  |  7   |
|     1      |   80711   | 0.04345447861629983  |  8   |
|     1      |   81225   | 0.04166666666666666  |  9   |
|     1      |   87962   | 0.022700748660347683 |  10  |
|     2      |   79595   |         0.5          |  1   |
|     2      |   49644   |         0.5          |  2   |
|     2      |   49653   |         0.5          |  3   |
|     2      |   48887   | 0.33333333333333337  |  4   |
|     2      |   54833   | 0.16

#### Observação
* Na filtragem colaborativa acima, usamos duas abordagens: cosseno e distância de Pearson. Também conseguimos aplicá-los a três conjuntos de dados de treinamento com contagens normais, dummy ou contagens normalizadas de compra de itens.
* Podemos ver que as recomendações são diferentes para cada usuário. Isso sugere que a personalização existe.
* Mas quão bom é este modelo em comparação com a linha de base e entre si? Precisamos de alguns meios de avaliar um mecanismo de recomendação. Vamos nos concentrar nisso na próxima seção.

## 7. Avaliação do modelo
Para avaliar os mecanismos de recomendação, podemos usar o conceito de recall de precisão.

* RMSE (erros quadráticos médios)
    * Mede o erro dos valores previstos
    * Quanto menor o valor RMSE, melhores as recomendações
* Lembrar
    * Qual porcentagem de produtos que um usuário compra são realmente recomendados?
    * Se um cliente comprar 5 produtos e a recomendação decidir mostrar 3 deles, o recall é de 0,6
* Precisão
    * De todos os itens recomendados, quantos o usuário realmente gostou?
    * Se 5 produtos foram recomendados ao cliente, dos quais ele compra 4 deles, a precisão é 0,8
    
* Por que o recall e a precisão são importantes?
    * Considere um caso em que recomendamos todos os produtos, para que nossos clientes com certeza tenha todos os itens que gostaria de compraram. Nesse caso, temos 100% de recall! Isso significa que nosso modelo é bom?
    * Temos que considerar a precisão. Se recomendarmos 300 itens, mas o usuário gostar e comprar apenas 3 deles, a precisão é de 0,1%! Essa precisão muito baixa indica que o modelo não é ótimo, apesar de seu excelente recall.
    * Portanto, nosso objetivo deve ser otimizar o recall e a precisão (para ser o mais próximo possível de 1).

Vamos comparar todos os modelos que construímos com base nas características de recuperação de precisão:

In [ ]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Modelos para contagem de compra

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/3192 queries. users per second: 14314.8

recommendations finished on 2000/3192 queries. users per second: 14200.3

recommendations finished on 3000/3192 queries. users per second: 14206


Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |     mean_precision     |      mean_recall      |
+--------+------------------------+-----------------------+
|   1    |          0.0           |          0.0          |
|   2    | 0.0001566416040100251  | 0.0003132832080200502 |
|   3    | 0.00010442773600668329 | 0.0003132832080200502 |
|   4    | 7.832080200501255e-05  | 0.0003132832080200502 |
|   5    | 6.265664160401006e-05  | 0.0003132832080200502 |
|   6    | 0.00010442773600668336 | 0.0006265664160401004 |
|   7    | 0.00031328320802004984 | 0.0019841269841269853 |
|   8    | 0.0003524436090225563  | 0.0022190893901420186 |
|   9    | 0.0004525201893622942  | 0.0031067251461988307 |
|   10   | 0.00043859649122807056 | 0.0034200083542188808 |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.06641231564123698

Per User RMSE (best)
+------------+------+-------+

...


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |  0.010338345864661662 | 0.008980785296574771 |
|   2    |   0.0081453634085213  | 0.013601712614870518 |
|   3    | 0.0061612364243943205 | 0.015403091060985806 |
|   4    |  0.005090852130325817 | 0.016917293233082702 |
|   5    |  0.004323308270676686 | 0.01801378446115288  |
|   6    |  0.003968253968253971 | 0.01999791144527988  |
|   7    |  0.003535624776226278 | 0.020937761069340033 |
|   8    |  0.003641917293233084 | 0.024331662489557235 |
|   9    |  0.003585352269562789 | 0.02689014202172098  |
|   10   | 0.0033834586466165344 | 0.028299916457811172 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.000914658220433

Per User RMSE (best)
+------------+--------------------+-------+
| customerId | 

recommendations finished on 1000/3192 queries. users per second: 10142.3

recommendations finished on 2000/3192 queries. users per second: 10087.7

recommendations finished on 3000/3192 queries. users per second: 10154.2


Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |     mean_precision     |      mean_recall      |
+--------+------------------------+-----------------------+
|   1    |          0.0           |          0.0          |
|   2    | 0.0001566416040100251  | 0.0003132832080200502 |
|   3    | 0.00010442773600668336 | 0.0003132832080200502 |
|   4    | 7.832080200501255e-05  | 0.0003132832080200502 |
|   5    | 6.265664160401008e-05  | 0.0003132832080200502 |
|   6    | 0.0002610693400167085  |  0.001566416040100251 |
|   7    | 0.0003132832080200501  | 0.0019841269841269853 |
|   8    | 0.0003524436090225563  |  0.002219089390142023 |
|   9    | 0.00045252018936229433 | 0.0031067251461988285 |
|   10   | 0.00043859649122807056 | 0.0034200083542188816 |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.46808719120957853

Per User RMSE (best)
+------------+------+-------+

#### Modelo de compra dummy

In [ ]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/3240 queries. users per second: 15087.3

recommendations finished on 2000/3240 queries. users per second: 14469

recommendations finished on 3000/3240 queries. users per second: 14543.7


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |          0.0          |          0.0           |
|   2    |          0.0          |          0.0           |
|   3    |          0.0          |          0.0           |
|   4    |          0.0          |          0.0           |
|   5    |          0.0          |          0.0           |
|   6    |          0.0          |          0.0           |
|   7    |          0.0          |          0.0           |
|   8    | 3.858024691358025e-05 |  0.000154320987654321  |
|   9    | 6.858710562414262e-05 | 0.00046296296296296255 |
|   10   | 6.172839506172834e-05 | 0.00046296296296296255 |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+------------+------+-------+
| customerId | 

...


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |  0.009876543209876543 | 0.00787037037037038  |
|   2    | 0.0060185185185185185 | 0.009567901234567909 |
|   3    |  0.005246913580246918 | 0.012577160493827162 |
|   4    |  0.004398148148148148 | 0.014274691358024706 |
|   5    |  0.00401234567901234  | 0.016085390946502055 |
|   6    |  0.003909465020576132 | 0.01861625514403292  |
|   7    | 0.0037477954144620805 | 0.021033950617283956 |
|   8    | 0.0034336419753086436 | 0.022062757201646112 |
|   9    | 0.0032578875171467713 |  0.0236059670781893  |
|   10   |  0.003148148148148145 | 0.025457818930041166 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9957143950870015

Per User RMSE (best)
+------------+--------------------+-------+
| customerId |

recommendations finished on 1000/3240 queries. users per second: 8594.24

recommendations finished on 2000/3240 queries. users per second: 8554.98

recommendations finished on 3000/3240 queries. users per second: 8756.9


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |          0.0          |         0.0          |
|   2    |          0.0          |         0.0          |
|   3    |          0.0          |         0.0          |
|   4    |          0.0          |         0.0          |
|   5    |          0.0          |         0.0          |
|   6    |          0.0          |         0.0          |
|   7    |          0.0          |         0.0          |
|   8    | 3.858024691358025e-05 | 0.000154320987654321 |
|   9    | 6.858710562414258e-05 | 0.000462962962962963 |
|   10   | 6.172839506172844e-05 | 0.000462962962962963 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------

#### Modelos de frequencia normalizado

In [ ]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |     mean_precision     |      mean_recall      |
+--------+------------------------+-----------------------+
|   1    |          0.0           |          0.0          |
|   2    |          0.0           |          0.0          |
|   3    |          0.0           |          0.0          |
|   4    |          0.0           |          0.0          |
|   5    | 0.00043383947939262476 | 0.0021691973969631237 |
|   6    | 0.0010845986984815619  | 0.0065075921908893716 |
|   7    | 0.0012395413696932134  |  0.008676789587852498 |
|   8    | 0.0029826464208242954  |  0.023861171366594363 |
|   9    |  0.003856350927934442  |  0.03470715835140997  |
|   10   |  0.02754880694143166   |   0.272234273318872   |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1

## 8. Seleção de modelo
### 8.1. Resumo da avaliação
* Com base em RMSE


    1. Popularidade nas contagens de compra: 1,27
    2. Similaridade de cosseno nas contagens de compra: 1,67
    3. Similaridade de Pearson nas contagens de compra: 1,27
    
    4. Popularidade no dummy de compra: 0
    5. Similaridade de cosseno no dummy de compra: 0,3
    6. Similaridade de Pearson no dummy de compra: 1
    
    7. Popularidade em contagens de compra em normalizada: 1,19
    8. Similaridade de cosseno em contagens de compra em normalizada:0,23
    9. Similaridade de Pearson nas contagens de compra em normalizada: 0,19
    
    
* Com base na precisão e recall
![](../images/model_comparisons.png)


#### Notas

* Popularidade v. Filtragem colaborativa: podemos ver que os algoritmos de filtragem colaborativa funcionam melhor do que o modelo de popularidade para contagens de compras. Na verdade, o modelo de popularidade não oferece nenhuma personalização, uma vez que apenas fornece a mesma lista de itens recomendados para todos os usuários.
* Precisão e recall: Observando o resumo acima, vemos que a precisão e o recall para Contagens de compra> Dummy de compra> Contagens de compra normalizadas. No entanto, como as pontuações de recomendação para os dados de compra normalizados são zero e constantes, escolhemos a dummy. Na verdade, o RMSE não é muito diferente entre os modelos do dummy e os dos dados normalizados.
* RMSE: Como o RMSE é maior usando a distância de Pearson do que o cosseno, escolheríamos modelar os erros quadrados médios menores, que neste caso seriam o cosseno.
Portanto, selecionamos a abordagem de similaridade de cosseno no dummy de compra como nosso modelo final.

## 8. Resultado final
* Nesta etapa, gostaríamos de manipular o formato da saída de recomendação para um que possamos exportar para csv e também uma função que retornará a lista de recomendações com um ID de cliente.
* Precisamos primeiro executar novamente o modelo usando todo o conjunto de dados, pois chegamos a um modelo final usando dados de trem e avaliados com conjunto de teste.

In [ ]:
#users_to_recommend = list(userId)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

len(users_to_recommend)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 20001 observations with 8982 users and 6910 items.

Data prepared in: 0.041176s

...

recommendations finished on 18000/20001 queries. users per second: 12853.4

recommendations finished on 19000/20001 queries. users per second: 12866.6

recommendations finished on 20000/20001 queries. users per second: 12747.7

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     1      |   96161   | 0.13437551259994507  |  1   |
|     1      |   100951  | 0.10867345333099365  |  2   |
|     1      |   96481   | 0.061429500579833984 |  3   |
|     1      |   88203   | 0.061429500579833984 |  4   |
|     1      |   72654   | 0.061429500579833984 |  5   |
|     1      |     2     | 0.061429500579833984 |  6   |
|     1      |   65540   | 0.05804544687271118  |  7   |
|     1      |   74235   | 0.05791628360748291  |  8   |
|     1      |   10437   | 0.05739426612854004  |  9   |
|     1      |     1     | 0.05319952964782715  |  10  |
|     2      |   96161   | 0.13437551259994507  |  1   |
|     2      |   100951  | 0.10867345333099365  |  2   |
|     2      |   96481   | 0.061429500579833984 |  3   |
|     2      |   88203   | 0.061429500579833984 |  4   |
|     2      |   72654   | 0.06

20001

### 8.1. CSV output file

In [ ]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head(30)

(200010, 4)


,customerId,productId,score,rank
0,1,96161,0.134376,1
1,1,100951,0.108673,2
2,1,96481,0.061430,3
3,1,88203,0.061430,4
4,1,72654,0.061430,5
5,1,2,0.061430,6
6,1,65540,0.058045,7
7,1,74235,0.057916,8
8,1,10437,0.057394,9
9,1,1,0.053200,10


In [ ]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define função para cria a saida desejada


In [ ]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('/content/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [ ]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/20001 queries. users per second: 362582

recommendations finished on 2000/20001 queries. users per second: 277546

recommendations finished on 3000/20001 queries. users per second: 211775

recommendations finished on 4000/20001 queries. users per second: 205149

recommendations finished on 5000/20001 queries. users per second: 207082

...

recommendations finished on 18000/20001 queries. users per second: 193430

recommendations finished on 19000/20001 queries. users per second: 193919

recommendations finished on 20000/20001 queries. users per second: 190092

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(8982, 1)


,recommendedProducts
customerId,
1,79595|49644|49653|48887|54833|80811|74095|8071...
2,79595|49644|49653|48887|54833|80811|74095|8071...
29,79595|49644|49653|48887|54833|80811|74095|8071...
203,79595|49644|49653|48887|54833|80811|74095|8071...
247,79595|49644|49653|48887|54833|80811|74095|8071...


### 8.2. Função de recomendação

In [ ]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [ ]:
customer_recomendation(1)

recommendedProducts    79595|49644|49653|48887|54833|80811|74095|8071...
Name: 1, dtype: object

In [ ]:
customer_recomendation(247)

recommendedProducts    79595|49644|49653|48887|54833|80811|74095|8071...
Name: 247, dtype: object

## Resumo
Neste exercício, pudemos realizar um processo passo a passo para fazer recomendações aos clientes. Usamos abordagens de Filtragem Colaborativa com as medidas `cosine` e` pearson` e comparamos os modelos com nosso modelo de popularidade de linha de base. Também preparamos três conjuntos de dados que incluem contagem de compras regular, compra dummy, bem como frequência de compra normalizada como nossa variável de destino. Usando RMSE, precisão e recall, avaliamos nossos modelos e observamos o impacto da personalização. Finalmente, selecionamos a abordagem Cosine em dados de compra dummy.